In [116]:
!pip uninstall openai
!pip install openai==0.28 huggingface-hub python-dotenv pandas requests

# Print OpenAI version
print(f"OpenAI version: {openai.__version__}")

In [ ]:

# Import required libraries
import os
import openai
import requests
import pandas as pd
from dotenv import load_dotenv
from huggingface_hub import InferenceApi

# Load environment variables
load_dotenv()

# Set API keys
openai.api_key = os.getenv("OPENAI_API_KEY")
proxycurl_api_key = os.getenv("PROXYCURL_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")
linkedin_client_id = os.getenv("LINKEDIN_CLIENT_ID")
linkedin_client_secret = os.getenv("LINKEDIN_CLIENT_SECRET")
linkedin_redirect_uri = os.getenv("LINKEDIN_REDIRECT_URI")
hf_api_key= os.getenv("HUGGING_FACE_API_KEY")

In [ ]:
# Initialize Hugging Face Inference API
hf = InferenceApi(repo_id="distilbert-base-uncased-finetuned-sst-2-english", token=hf_api_key)

# Function to extract text from profile sections
def extract_text_from_sections(profile, sections):
    text = ''
    if 'bio' in sections and profile.get('summary'):
        text += profile['summary'] + ' '
    if 'projects' in sections and profile.get('projects'):
        for project in profile['projects']:
            if project.get('description'):
                text += project['description'] + ' '
    if 'experience' in sections and profile.get('experiences'):
        for experience in profile['experiences']:
            if experience.get('description'):
                text += experience['description'] + ' '
    if 'skills' in sections and profile.get('skills'):
        text += ' '.join(profile['skills']) + ' '
    return text.strip()

In [ ]:
# Function to analyze traits using OpenAI
def analyze_traits(profile):
    try:
        text = extract_text_from_sections(profile, ['bio', 'projects', 'experience'])
        print('Analyzing traits with text:', text)
        prompt = f"Analyze the following text and list the top 7 traits that describe the person professionally: {text}"
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}]
        )
        traits = response.choices[0].message['content'].split('\n').filter(Boolean)
        print('Generated Traits:', traits)
        return traits
    except Exception as e:
        print('Error analyzing traits:', str(e))
        return []


In [ ]:
# Function to generate ice breakers using OpenAI
def generate_ice_breakers(profile):
    try:
        prompt = f"Generate 3 unique and engaging ice breakers based on this LinkedIn profile. The ice breakers should be professional, related to their work experience or skills, and encourage meaningful conversation. Profile: {profile}"
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}]
        )
        ice_breakers = response.choices[0].message['content'].split('\n').filter(Boolean)
        print('Generated Ice Breakers:', ice_breakers)
        return ice_breakers
    except Exception as e:
        print('Error generating ice breakers:', str(e))
        return []


In [ ]:
# Function to generate tech skills using OpenAI
def generate_tech_skills(profile):
    try:
        text = extract_text_from_sections(profile, ['bio', 'projects', 'experience'])
        print('Generating tech skills with text:', text)
        prompt = f"Analyze the following text and list the top 10 technical skills that the person possesses: {text}"
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}]
        )
        tech_skills = response.choices[0].message['content'].split('\n').filter(Boolean)
        print('Generated Tech Skills:', tech_skills)
        return tech_skills
    except Exception as e:
        print('Error generating tech skills:', str(e))
        return []


In [ ]:
# Function to enhance skills with emotion analysis using Hugging Face
def enhance_skills_with_emotion_analysis(skills):
    try:
        enhanced_skills = []
        for skill in skills:
            result = hf(inputs=skill)
            print('Emotion Analysis Result for skill:', skill, result)
            enhanced_skills.append(f"{skill} ({result[0]['label']}: {result[0]['score']:.2f})")
        return enhanced_skills
    except Exception as e:
        print('Error enhancing skills with emotion analysis:', str(e))
        return skills

In [ ]:
# Function to generate job contributions using OpenAI
def generate_job_contributions(profile):
    try:
        prompt = f"Generate 5 significant job contributions based on this LinkedIn profile. The contributions should highlight the impact and achievements in their roles. Profile: {profile}"
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}]
        )
        job_contributions = response.choices[0].message['content'].split('\n')
        job_contributions = [job_contribution.strip() for job_contribution in job_contributions if job_contribution.strip()]
        print('Generated Job Contributions:', job_contributions)
        return job_contributions
    except Exception as e:
        print('Error generating job contributions:', str(e))
        return []

In [ ]:
# Function to fetch LinkedIn profile using ProxyCurl
def get_linkedin_profile_by_url(linkedin_url):
    try:
        response = requests.get(
            'https://nubela.co/proxycurl/api/v2/linkedin',
            params={'url': linkedin_url},
            headers={'Authorization': f'Bearer {proxycurl_api_key}'}
        )
        return process_proxycurl_data(response.json())
    except Exception as e:
        print('Error fetching LinkedIn data from ProxyCurl:', str(e))
        return {}

In [ ]:
# Function to process ProxyCurl data
def process_proxycurl_data(data):
    return {
        'name': f"{data.get('first_name')} {data.get('last_name')}",
        'headline': data.get('headline'),
        'summary': data.get('summary'),
        'profileUrl': f"https://www.linkedin.com/in/{data.get('public_identifier')}/" if data.get('public_identifier') else None,
        'location': data.get('location'),
        'industry': data.get('industry'),
        'experiences': [{'title': exp.get('title'), 'company': exp.get('company'), 'startDate': exp.get('starts_at'), 'endDate': exp.get('ends_at'), 'description': exp.get('description')} for exp in data.get('experiences', [])],
        'education': [{'school': edu.get('school'), 'degree': edu.get('degree_name'), 'fieldOfStudy': edu.get('field_of_study'), 'startDate': edu.get('starts_at'), 'endDate': edu.get('ends_at')} for edu in data.get('education', [])],
        'skills': data.get('skills', [])
    }

In [ ]:
# Function to search LinkedIn profile using Tavily
def search_linkedin_profile(name):
    try:
        response = requests.get(
            'https://api.tavily.com/search',
            params={
                'api_key': tavily_api_key,
                'query': f'LinkedIn profile of {name}',
                'search_depth': 'advanced',
                'include_domains': 'linkedin.com'
            }
        )
        return process_linkedin_data(response.json())
    except Exception as e:
        print('Error fetching LinkedIn data from Tavily:', str(e))
        return {}


In [ ]:
# Function to process LinkedIn data from Tavily
def process_linkedin_data(tavily_response):
    linkedIn_result = next((result for result in tavily_response.get('results', []) if 'linkedin.com/in/' in result.get('url', '')), None)
    if not linkedIn_result:
        raise ValueError('LinkedIn profile not found')
    return {
        'name': linkedIn_result['title'].split(' | ')[0],
        'headline': linkedIn_result['title'].split(' | ')[1],
        'summary': linkedIn_result['snippet'],
        'profileUrl': linkedIn_result['url']
    }

In [ ]:
# Sample LinkedIn profile data for testing
sample_profile = {
    'summary': 'Senior Architect at Google with 20 years of experience in software engineering, cloud architecture, and AI development. Demonstrated expertise in leading large-scale projects, driving innovation, and mentoring engineering teams.',
    'projects': [
        {'description': 'Led the design and implementation of Google Cloud’s next-generation infrastructure, improving scalability and performance by 30%.'},
        {'description': 'Architected a machine learning platform used by over 1,000 data scientists to develop and deploy AI models at scale.'},
        {'description': 'Spearheaded the migration of legacy systems to a microservices architecture, resulting in a 40% reduction in system downtime.'}
    ],
    'experiences': [
        {
            'title': 'Senior Architect',
            'company': 'Google',
            'startDate': {'year': 2004},
            'endDate': {'year': 2024},
            'description': 'Oversaw the architecture of Google Cloud services, ensuring high availability, security, and efficiency. Mentored a team of 50+ engineers and collaborated with cross-functional teams to deliver cutting-edge solutions.'
        },
        {
            'title': 'Principal Software Engineer',
            'company': 'Google',
            'startDate': {'year': 2000},
            'endDate': {'year': 2004},
            'description': 'Developed core components of Google Search and Google Ads, optimizing algorithms for better performance and accuracy. Led the adoption of new technologies and practices within the engineering department.'
        }
    ],
    'education': [
        {
            'school': 'Stanford University',
            'degree': 'Master of Science',
            'fieldOfStudy': 'Computer Science',
            'startDate': {'year': 1998},
            'endDate': {'year': 2000}
        },
        {
            'school': 'University of California, Berkeley',
            'degree': 'Bachelor of Science',
            'fieldOfStudy': 'Electrical Engineering and Computer Science',
            'startDate': {'year': 1994},
            'endDate': {'year': 1998}
        }
    ],
    'skills': ['Cloud Architecture', 'Machine Learning', 'Microservices', 'Kubernetes', 'Distributed Systems', 'Artificial Intelligence', 'Python', 'Go', 'Java', 'Team Leadership', 'Project Management']
}


In [ ]:
# Test functions
traits = analyze_traits(sample_profile)
print("Traits:", traits)

tech_skills = generate_tech_skills(sample_profile)
print("Tech Skills:", tech_skills)

enhanced_tech_skills = enhance_skills_with_emotion_analysis(tech_skills)
print("Enhanced Tech Skills:", enhanced_tech_skills)

job_contributions = generate_job_contributions(sample_profile)
print("Job Contributions:", job_contributions)

# Fetch and display LinkedIn profile data from ProxyCurl
linkedin_profile = get_linkedin_profile_by_url('https://www.linkedin.com/in/sample-profile/')
print("LinkedIn Profile (ProxyCurl):", linkedin_profile)

# Search and display LinkedIn profile data from Tavily
searched_profile = search_linkedin_profile('Sample Name')
print("Searched LinkedIn Profile (Tavily):", searched_profile)

Analyzing traits with text: Senior Architect at Google with 20 years of experience in software engineering, cloud architecture, and AI development. Demonstrated expertise in leading large-scale projects, driving innovation, and mentoring engineering teams. Led the design and implementation of Google Cloud’s next-generation infrastructure, improving scalability and performance by 30%. Architected a machine learning platform used by over 1,000 data scientists to develop and deploy AI models at scale. Spearheaded the migration of legacy systems to a microservices architecture, resulting in a 40% reduction in system downtime. Oversaw the architecture of Google Cloud services, ensuring high availability, security, and efficiency. Mentored a team of 50+ engineers and collaborated with cross-functional teams to deliver cutting-edge solutions. Developed core components of Google Search and Google Ads, optimizing algorithms for better performance and accuracy. Led the adoption of new technologi

LinkedIn Profile (ProxyCurl): {'name': 'Sample Profile', 'headline': 'Medical Assistant who helps treat and encourage patients. Certified through the American Medical Technologists.--', 'summary': 'My passion for caring and positively influencing people led me to choose a career in the medical field, specifically medical assisting. I believe that compassion is key and treating each person with respect and dignity will allow me to give the best possible service I can.\n\nI believe that advancement in the workforce comes with good experience and I am looking for an entry level medical assistant position with an office that will give me great experience to help me advance my skills and knowledge in the medical field. I believe that a positive, encouraging work environment leads to successful employees, and I would love to be part of such a team. \n\nI will be completing my extern hours at such and such clinic from January 2017-March 2017. I am excited to get first hand experience in a med